<a href="https://colab.research.google.com/github/vinithreddybanda/dl2/blob/main/DL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install datasets

In [29]:
# 1. Load Required Libraries
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam

# 2. Load and Clean the Dataset
train_df = pd.read_csv('/content/hi.translit.sampled.train.tsv', sep='\t', header=None, encoding='utf-8')
test_df = pd.read_csv('/content/hi.translit.sampled.test.tsv', sep='\t', header=None, encoding='utf-8')
dev_df = pd.read_csv('/content/hi.translit.sampled.dev.tsv', sep='\t', header=None, encoding='utf-8')

# Drop unwanted third column and rename
train_df = train_df.drop(columns=[2])
test_df = test_df.drop(columns=[2])
dev_df = dev_df.drop(columns=[2])

train_df.columns = ['devanagari', 'latin']
test_df.columns = ['devanagari', 'latin']
dev_df.columns = ['devanagari', 'latin']

# Drop NaNs and convert to strings
for df in [train_df, test_df, dev_df]:
    df.dropna(inplace=True)
    df['latin'] = df['latin'].astype(str)
    df['devanagari'] = df['devanagari'].astype(str)

print("Train Sample:\n", train_df.head())

# 3. Tokenize the Texts
latin_tokenizer = Tokenizer(char_level=True)
latin_tokenizer.fit_on_texts(train_df['latin'])

devanagari_tokenizer = Tokenizer(char_level=True)
devanagari_tokenizer.fit_on_texts(train_df['devanagari'])

input_vocab_size = len(latin_tokenizer.word_index) + 1
output_vocab_size = len(devanagari_tokenizer.word_index) + 1

print(f"Latin vocab size: {input_vocab_size}")
print(f"Devanagari vocab size: {output_vocab_size}")

# 4. Prepare the Data for Training
train_input_seq = latin_tokenizer.texts_to_sequences(train_df['latin'])
train_output_seq = devanagari_tokenizer.texts_to_sequences(train_df['devanagari'])

max_input_length = max(len(seq) for seq in train_input_seq)
max_output_length = max(len(seq) for seq in train_output_seq)

train_input_seq = pad_sequences(train_input_seq, maxlen=max_input_length, padding='post')
train_output_seq = pad_sequences(train_output_seq, maxlen=max_output_length, padding='post')

train_output_seq_one_hot = np.array([
    to_categorical(seq, num_classes=output_vocab_size)
    for seq in train_output_seq
])

print(f"Train Input Shape: {train_input_seq.shape}")
print(f"Train Output Shape (one-hot): {train_output_seq_one_hot.shape}")

# 5. Define the Seq2Seq Model
def define_model(input_vocab_size, output_vocab_size, input_timesteps, output_timesteps, embedding_dim, hidden_units):
    encoder_inputs = Input(shape=(input_timesteps,))
    encoder_embedding = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
    encoder_outputs, state_h, state_c = LSTM(hidden_units, return_state=True)(encoder_embedding)
    encoder_states = [state_h, state_c]

    decoder_inputs = Input(shape=(output_timesteps,))
    decoder_embedding = Embedding(output_vocab_size, embedding_dim)(decoder_inputs)
    decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
    decoder_dense = Dense(output_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

embedding_dim = 50
hidden_units = 256
model = define_model(input_vocab_size, output_vocab_size, max_input_length, max_output_length, embedding_dim, hidden_units)
model.summary()

# 6. Train the Model
decoder_input_train = np.zeros_like(train_output_seq)
decoder_input_train[:, 1:] = train_output_seq[:, :-1]

model.fit(
    [train_input_seq, decoder_input_train],
    train_output_seq_one_hot,
    batch_size=64,
    epochs=50,
    validation_split=0.2
)

# 7. Evaluate the Model
test_input_seq = latin_tokenizer.texts_to_sequences(test_df['latin'])
test_output_seq = devanagari_tokenizer.texts_to_sequences(test_df['devanagari'])

test_input_seq = pad_sequences(test_input_seq, maxlen=max_input_length, padding='post')
test_output_seq = pad_sequences(test_output_seq, maxlen=max_output_length, padding='post')

decoder_input_test = np.zeros_like(test_output_seq)
decoder_input_test[:, 1:] = test_output_seq[:, :-1]

test_output_seq_one_hot = np.array([
    to_categorical(seq, num_classes=output_vocab_size)
    for seq in test_output_seq
])

test_loss, test_acc = model.evaluate([test_input_seq, decoder_input_test], test_output_seq_one_hot)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# 8. Make Predictions
def decode_sequence(input_seq):
    decoder_input = np.zeros((1, max_output_length))
    for i in range(max_output_length):
        output_tokens = model.predict([input_seq, decoder_input], verbose=0)
        sampled_token_index = np.argmax(output_tokens[0, i, :])
        decoder_input[0, i] = sampled_token_index
    predicted_seq = decoder_input[0]
    predicted_text = ''.join([devanagari_tokenizer.index_word.get(int(i), '') for i in predicted_seq if i != 0])
    return predicted_text

# Predict on one test sample
sample_input = test_input_seq[0:1]
predicted_output = decode_sequence(sample_input)
print(f"\nLatin Input       : {test_df.iloc[0]['latin']}")
print(f"Actual Devanagari : {test_df.iloc[0]['devanagari']}")
print(f"Predicted Output  : {predicted_output}")


Train Sample:
   devanagari     latin
0         अं        an
1    अंकगणित  ankganit
2       अंकल     uncle
3      अंकुर     ankur
4     अंकुरण   ankuran
Latin vocab size: 27
Devanagari vocab size: 64
Train Input Shape: (44202, 20)
Train Output Shape (one-hot): (44202, 19, 64)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 19)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 20, 50)    │      1,350 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 19, 50)    │      3,200 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, 256),     │    314,368 │ embedding[0][0]   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, 19, 256), │    314,368 │ embedding_1[0][0… │
│                     │ (None, 256),      │            │ lstm_2[0][1],     │
│                     │ (None, 256)]      │            │ lstm_2[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 19, 64)    │     16,448 │ lstm_3[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 649,734 (2.48 MB)

 Trainable params: 649,734 (2.48 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.6909 - loss: 1.3509 - val_accuracy: 0.7070 - val_loss: 1.2421
Epoch 2/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.7429 - loss: 0.9304 - val_accuracy: 0.7236 - val_loss: 1.2489
Epoch 3/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.7773 - loss: 0.7808 - val_accuracy: 0.7374 - val_loss: 1.2211
Epoch 4/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.8157 - loss: 0.6324 - val_accuracy: 0.7526 - val_loss: 1.1188
Epoch 5/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.8526 - loss: 0.4931 - val_accuracy: 0.7785 - val_loss: 0.9815
Epoch 6/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.8894 - loss: 0.3625 - val_accuracy: 0.8003 - val_loss: 0.8598
Epoch 7/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9141 - loss: 0.2799 - val_accuracy: 0.8097 - val_loss: 0.8153
Epoch 8/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9291 - loss: 0.2290 - val_acc

In [13]:
import pandas as pd
import re
from datasets import Dataset
from transformers import (
    GPT2Tokenizer, GPT2LMHeadModel,
    Trainer, TrainingArguments,
    DataCollatorForLanguageModeling,
    pipeline, set_seed
)

# ---------- Step 1: Load and Clean Lyrics ----------
def clean_lyrics(lyric):
    if pd.isna(lyric):
        return ""
    lyric = re.sub(r'^#+', '', str(lyric))
    lyric = lyric.encode('utf-8').decode('utf-8', 'ignore')
    lyric = re.sub(r'[\u2018\u2019\u201c\u201d]+', "'", lyric)
    lyric = re.sub(r'[^\x00-\x7F]+', '', lyric)
    return lyric.strip()

def load_and_clean_lyrics(file_paths):
    df = pd.concat([pd.read_csv(path) for path in file_paths], ignore_index=True)
    if 'Lyric' not in df.columns:
        raise ValueError("Missing 'Lyric' column.")
    df['Lyric'] = df['Lyric'].astype(str).apply(clean_lyrics)
    return df['Lyric'].dropna().tolist()

# ---------- Step 2: Tokenizer and Dataset ----------
def get_tokenizer_and_model(model_name="gpt2"):
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    model = GPT2LMHeadModel.from_pretrained(model_name)
    model.config.pad_token_id = tokenizer.pad_token_id
    return tokenizer, model

def prepare_dataset(lyrics, tokenizer, max_length=512):
    dataset = Dataset.from_dict({"text": lyrics})

    def tokenize(example):
        return tokenizer(
            example["text"],
            truncation=True,
            padding="max_length",
            max_length=max_length
        )

    dataset = dataset.map(tokenize, batched=True)
    dataset = dataset.map(lambda x: {"labels": x["input_ids"]}, batched=True)
    return dataset

# ---------- Step 3: Train ----------
def train_model(dataset, model, tokenizer):
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    training_args = TrainingArguments(
        output_dir="./gpt2-lyrics-output",
        per_device_train_batch_size=2,
        num_train_epochs=3,
        logging_steps=50,
        save_steps=500,
        save_total_limit=1,
        report_to="none",
        prediction_loss_only=True,
        fp16=False  # Set True if using GPU w/ float16 support
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        tokenizer=tokenizer,
        data_collator=data_collator
    )

    trainer.train()

# ---------- Step 4: Generate ----------
def generate_lyrics(prompt="Under the neon lights, we danced"):
    tokenizer, model = get_tokenizer_and_model()
    generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
    set_seed(42)
    output = generator(prompt, max_length=100, num_return_sequences=1)
    print("\n🎶 Lyrics\n")
    print(output[0]["generated_text"])

# ---------- Main ----------
if __name__ == "__main__":
    file_paths = ["/content/ArianaGrande.csv", "/content/BillieEilish.csv"]
    lyrics = load_and_clean_lyrics(file_paths)
    tokenizer, model = get_tokenizer_and_model()
    dataset = prepare_dataset(lyrics, tokenizer)
    train_model(dataset, model, tokenizer)
    generate_lyrics(prompt="Under the neon lights, we danced")


Map:   0%|          | 0/453 [00:00<?, ? examples/s]

Map:   0%|          | 0/453 [00:00<?, ? examples/s]

<ipython-input-13-a3eb8d2522ed>:67: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,2.935600
100,2.784300
150,2.687600
200,2.645800
250,2.573400
300,2.506000
350,2.298000
400,2.253000
450,2.349300
500,2.312500


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



🎶 Lyrics

Under the neon lights, we danced for the rest of the night, and they had the good fortune to see it fall on them instead of the rest of the crowd. After dancing for one minute, each time everyone seemed to be doing a different thing from him. Finally, a guy on stage started talking about his friend's parents — maybe something along the lines of: "We lost the boy at the hands of our dad in his thuggish way. I was right — I was right. He
